# Basic use of scikit-learn

## Loading pre-formated data

In [ ]:
# load the IRIS dataset
from sklearn.datasets import load_iris
irisData=load_iris()
# get info on the dataset
#print(irisData.data)
print(irisData.target)
print(irisData.target_names)
print(irisData.feature_names)
#print(irisData.DESCR)

**Q1:** What type of machine learning problem is that?

It's a multi-class classification problem.

**Q2:** How many features are there? What kind of features?

There are 4 features. They are all numerical.

## Plotting parts of the data

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt # replace the name "pyplot" by "plt" 
X=irisData.data
y=irisData.target
xi=0
yi=1

colors=["red","green","blue"] # associate a color to each class label
for num_label in range(3): # for each label
        plt.scatter(X[y==num_label][:, xi],X[y==num_label][:,yi],color=colors[num_label],label=irisData.target_names[num_label])
plt.legend()
plt.xlabel(irisData.feature_names[xi]) 
plt.ylabel(irisData.feature_names[yi])
plt.title("Iris Data - size of the sepals only") 
plt.show()

**Q3:** From the previous visualisation, what can you predict about the difficulty of this dataset?

Separating the blue and green classes will be hard because they are mixed.

## Classifying with kNN

In [ ]:
from sklearn import neighbors
nb_neighb = 15
clf = neighbors.KNeighborsClassifier(nb_neighb) # to know more about the parameters, type help(neighbors.KNeighborsClassifier)

clf.fit(X, y) # training
print('accuracy on X is',clf.score(X,y))

# to predict on a specific example
print('class predicted is',clf.predict([[ 5.4, 3.2, 1.6, 0.4]]))
print('proba of each class is',clf.predict_proba([[ 5.4, 3.2, 1.6, 0.4]]))

y_pred = clf.predict(X)
print('misclassified training examples are:',X[y_pred!=y])

**Q4:** What kind of problem do you see with the evaluation?

The model is evaluated on the training set.

## About training and test sets

if we want a test set and a training set, we can split the data

In [ ]:
X_train, y_train = X[0:100], y[0:100] # 100 examples for training
X_test, y_test = X[100:], y[100:] # rest for testing

**Q5:** Explain why it is a really bad idea to split this iris dataset as we've done.

The classes are not equally distributed in the training and test sets.

here is a much better way to split the data into training and test sets

In [ ]:
from sklearn.model_selection import train_test_split 
import random # to generate random numbers

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=random.seed()) # if needed: help(train_test_split)
print('size of train / test = ',len(X_train), len(X_test))
print('nb of training data with class 0/1/2 =', len(X_train[y_train==0]) ,len(X_train[y_train==1]), len(X_train[y_train==2]))

clf=clf.fit(X_train, y_train)
y_pred =clf.predict(X_test)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print('Confusion matrix\n',cm)


**Q6:** What is on the diagonal of the confusion matrix?

The number of good predictions for each class.

**Q7:** What is the real error rate (give details)?

The error rate is the number of errors divided by the total number of tests.
Here it's 3/45.

One can prefer cross-fold validation

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

nb_folds = 10
kf=KFold(n_splits=nb_folds,shuffle=True)
score=0
for training_ind,test_ind in kf.split(X):
    #print("training index: ",training_ind,"\ntest index:",test_ind,'\n') 
    X_train=X[training_ind]
    y_train=y[training_ind]
    clf.fit(X_train, y_train)
    X_test=X[test_ind]
    y_test=y[test_ind]
    y_pred = clf.predict(X_test)
    score = score + accuracy_score(y_pred,y_test)
print('average accuracy:',score/nb_folds)

or as a one-liner:

In [ ]:
from sklearn.model_selection import cross_val_score
t_scores = cross_val_score(clf, X, y, cv=10)
print(t_scores.mean())

# Decision tree

In [ ]:
# we will use another dataset (a CSV file). Pandas helps us to read this type of file.

import pandas as pd

data = 'heart.csv'
df = pd.read_csv(data)


X = df.drop(columns=['target'])
y = df['target']


X_train,X_test,y_train,y_test = train_test_split(X,y,stratify=y)

features = X.columns
classes = ['Not heart disease','heart disease']

print (features)

df.head()

In [ ]:
from sklearn import tree
from graphviz import Source

X_train,X_val,y_train,y_val=train_test_split(X,y,test_size=0.3,random_state=42)

clf = tree.DecisionTreeClassifier(max_depth=20,criterion='entropy')
clf.fit(X_train, y_train)

graph = Source(tree.export_graphviz(clf, out_file=None,
                                    feature_names=features,
                                    class_names=classes,
                                    filled=True, rounded=True))
graph

If Graphviz is not working with your setup, look at http://people.irisa.fr/Vincent.Claveau/cours/fd/TP1.html

**Q8:** Explain each line displayed in the nodes/leaves of the tree.

The first line is the criteria used to split the classes.
The second line is the entropy of the node.
The third line is the number of samples in the node.
The fourth line is the number of samples in each class (here there are two classes).
The fifth line is the class that is predicted if the node is a leaf.
    
**Q9:** What is the name of this decision tree according to the course?

It's a binary classification tree.


Here is another nice viz of the decision tree. (The dtreeviz package is available in github. It can be installed with 'pip install dtreeviz'. It requires graphviz to be installed.)

In [ ]:
from dtreeviz import * # remember to load the package

graph = model(clf, X_train, y_train,
                target_name="target",
                feature_names=features,
                class_names=classes
                )

graph.view()


**Q10:** Explain what are the histograms displayed.

The histograms show the distribution of the classes in a node on the node's criteria. 
The color of the bars is the class that is predicted if the node is a leaf. 


**Q11** From the sklearn manual, explain what effectmax_depth or min_samples_split will have on the decision tree. If time permits, show the effects experimentally.

- **max_depth**: The maximum depth of the tree.
- **min_samples_split**: The minimum number of samples required to split an internal node.


### Pruning Tmax

(from https://scikit-learn.org/stable/auto_examples/tree/plot_cost_complexity_pruning.html)

Here, we use a critrion called "Cost Complexity". Cost complexity pruning is all about finding the right parameter for alpha.We will get the alpha values for this tree and will check the accuracy with the pruned trees.

In [ ]:
path = clf.cost_complexity_pruning_path(X_train, y_train)
ccp_alphas, impurities = path.ccp_alphas, path.impurities
print(ccp_alphas)

In [ ]:
# For each alpha we will append our model to a list
t_clf = []
for ccp_alpha in ccp_alphas:
    clf = tree.DecisionTreeClassifier(random_state=0, ccp_alpha=ccp_alpha)
    clf.fit(X_train, y_train)
    t_clf.append(clf)
    
# we remove the last element in clfs and ccp_alphas, because it is the trivial tree with only one node.
t_clf = t_clf[:-1]
ccp_alphas = ccp_alphas[:-1]
node_counts = [clf.tree_.node_count for clf in t_clf]
depth = [clf.tree_.max_depth for clf in t_clf]
plt.scatter(ccp_alphas,node_counts)
plt.scatter(ccp_alphas,depth)
plt.plot(ccp_alphas,node_counts,label='no of nodes',drawstyle="steps-post")
plt.plot(ccp_alphas,depth,label='depth',drawstyle="steps-post")
plt.legend()
plt.title('Tree complexity vs alpha')
plt.show()


# accuracy versus alpha
train_acc = []
val_acc = []
for c in t_clf:
    y_train_pred = c.predict(X_train)
    y_val_pred = c.predict(X_val)
    train_acc.append(accuracy_score(y_train_pred,y_train))
    val_acc.append(accuracy_score(y_val_pred,y_val))

plt.scatter(ccp_alphas,train_acc)
plt.scatter(ccp_alphas,val_acc)
plt.plot(ccp_alphas,train_acc,label='train_accuracy',drawstyle="steps-post")
plt.plot(ccp_alphas,val_acc,label='val_accuracy',drawstyle="steps-post")
plt.legend()
plt.title('Accuracy vs alpha')
plt.show()

**Q12:** from the graph above, what is the best value for alpha. Replace it in the first line below .

In [ ]:
best_alpha = 0.02
clf_ = tree.DecisionTreeClassifier(random_state=0,ccp_alpha=best_alpha)
clf_.fit(X_train,y_train)
y_train_pred = clf_.predict(X_train)
y_val_pred = clf_.predict(X_val)

print('Train score', accuracy_score(y_train_pred,y_train))
print(confusion_matrix(y_train_pred,y_train))

print('Validation score', accuracy_score(y_val_pred,y_val))
print(confusion_matrix(y_val_pred,y_val))

# Bayesian learning

Scikit learn implements several variants of Bayesian learning, based on different assumptions about the data https://scikit-learn.org/stable/modules/naive_bayes.html

In [ ]:
data = 'weather.nominal.csv'
df = pd.read_csv(data)

df.head()

**Q13:** Let us consider the weather_nominal dataset. What is the type of each feature?

The features are all nominal.

In [ ]:
X_train = df.drop(columns=['play'])
y_train = df['play']

features = X_train.columns
classes = ['no play','play']

# we must convert the nominal features into integers
from sklearn.preprocessing import OrdinalEncoder
enc = OrdinalEncoder()
X_train = enc.fit_transform(X_train)


In [ ]:
from sklearn.naive_bayes import CategoricalNB

clf = CategoricalNB().fit(X_train,y_train)

y_train_pred = clf.predict(X_train)

print('Train score',accuracy_score(y_train_pred,y_train))
print(confusion_matrix(y_train_pred,y_train))

**Q14:** Explain what is displayed by the two following code lines and link that with what you've seen during the course. Do these figures correspond to what you get when doing it by yourself (explain)?

The first line displays the prior probabilities *(probabilités à priori)* of the classes. 
The second line displays the conditional probabilities of the features given the classes. 


In [ ]:
print(clf.class_log_prior_)
print(clf.feature_log_prob_)

**Q15:** Let's consider the weather.csv dataset now. Explain what is the difference with the previous one. 

Some features are numerical : the temperature and the humidity.

**Q16:** Compute 'by hand' the a posteriori proba of each class for the following data sample P(play=0|x=\['sunny',73,81,'TRUE'\]) and  P(play=1|x=\['sunny',73,81,'TRUE'\]) :

In [ ]:
file = 'weather.csv'
df = pd.read_csv(file)

df.head()

In [ ]:
X_train = df.drop(columns=['play'])
y_train = df['play']

X_test = [ ['sunny',73,81,'TRUE'] ]

features = X_train.columns
classes = ['no play','play']

# TODO : Fix the errors
enc = OrdinalEncoder()
X_train[['outlook','windy']] = enc.fit_transform(X_train[['outlook','windy']])

clf = CategoricalNB().fit(X_train,y_train)

y_train_pred = clf.predict(X_train)

print('Train score',accuracy_score(y_train_pred,y_train))
print(confusion_matrix(y_train_pred,y_train))



print('proba of each class is',clf.predict_proba(enc.transform(X_test)))